# CEM Workflow Notebook (MNG Version)
### Coastal Group, ESPIn 2021 Group Project
Created by Megan Gillen; 6-14-21
<br>Last Updated: 6-15-21

### 1. Import Relevant Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pymt.models import Cem, Waves
from IPython.display import clear_output

### 2. Initialize Models from PyMT
We are using the [Coastal Evolution Model (CEM)](https://csdms.colorado.edu/wiki/Model_help:CEM) and the [WAVES](https://csdms.colorado.edu/wiki/Model_help:Waves) models. The CEM simulates morphodynamic evolution of coastlines under varying wave climates. WAVES is the model that calculates the input wave energetics used in CEM as a function of wave period (T), deep water wave height (H0), and wave angle criteria.

In [2]:
cem = Cem()
waves = Waves()

### WAVES Model Input Parameters
- `sea_surface_water_wave__height` --> Deep water wave height (H0)
- `sea_surface_water_wave__period` --> Wave period (T)
- `sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_highness_parameter` --> Proportion of high angle waves (U)
> "The variable U controls the general directional spread of the approaching waves, here split into whether waves approach from angles great than or less than the one which maximized alongshore sediment transport (approximately 45 degrees)...U varies between 0-1, controls the fraction of high-angle waves, where a value of less than 0.5 indicates wave energy predominately approaching from a low angle and a designation greater than 0.5 indicates a predominance of high-angle (unstable waves)."
> "This variable is probably the most important control on the behavior of coupled CEM-WAVES simulations using this simplified wave climate scheme. If CEM is being employed to simulate the self-organization of a coast, values of U larger than 0.5 should be used. For scenarios involving delta evolution, values less than 0.5 tend to be more reasonable (unless a local wave climate has that character). There is no specific method for deriving these terms (U and A) from natural wave climate conditions. Either A and U can be toggled to emulate a natural wave or other methods can be employed (not yet functional) where a natural wave climate can be enetered into the model." 
- `sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_asymmetry_parameter` --> Asymmetry of wave direction (A)
> "The variable A controls the directional distribution of waves, representing the fraction of waves approaching from the left or right, from the perspective of a person looking out to sea. It is designated to be a value between 0 and 1. A designation of greater than 0.5 indicates that the majority of wave energy is approaching from the left where a designation of 1.0 indicates all wave energy approaches from the left. A designation of 0.5 indicates wave energy approach is evenly distributed between the left and right. A designation of less than 0.5 indicates the majority of wave energy is approaching from the right where a designation of 0 indicates all wave energy approaches from the right."

[WAVES Documentation](https://csdms.colorado.edu/wiki/Model_help:Waves)

In [3]:
# rename long variables
H0 = 'sea_surface_water_wave__height'
T = 'sea_surface_water_wave__period'
U = 'sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_highness_parameter'
A = 'sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_asymmetry_parameter'

### WAVES Model Output Parameters
- `sea_surface_water_wave__min_of_increment_of_azimuth_angle_of_opposite_of_phase_velocity` --> 
- `sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity` --> 
- `sea_surface_water_wave__mean_of_increment_of_azimuth_angle_of_opposite_of_phase_velocity` --> 
- `sea_surface_water_wave__max_of_increment_of_azimuth_angle_of_opposite_of_phase_velocity` --> 
- `sea_surface_water_wave__height` --> 
- `sea_surface_water_wave__period` --> 

In [4]:
alpha = 'sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity'

In [5]:
Qs = 'land_surface_water_sediment~bedload__mass_flow_rate'
Z = 'sea_water__depth'
dt = 'model__time_step'

In [42]:
args = cem.setup(number_of_rows=100, number_of_cols=200, grid_spacing=200.)

waves.initialize(*waves.setup())
cem.initialize(*args)

In [43]:
grid_id = cem.var_grid(Z)
spacing = cem.grid_spacing(grid_id)
shape = cem.grid_shape(grid_id)

In [44]:
waves.set_value(H0, 2.5);
waves.set_value(T, 7.);
waves.set_value(U, 1);
waves.set_value(A, 1);

z = np.empty(shape);
cem.get_value(Z, out=z);
qs = np.zeros_like(z);
qs[0, 100] = 3050;
cem.set_value(Qs,qs);

In [45]:
cem.input_var_names

('sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity',
 'land_surface_water_sediment~bedload__mass_flow_rate',
 'sea_surface_water_wave__period',
 'sea_surface_water_wave__height',
 'land_surface__elevation',
 'model__time_step')

## --------------Conner edits/additions---------------------

In [46]:
def plot_coast(z):
    fig,ax = plt.subplots(figsize=(12,5))
    im = ax.imshow(z, origin='lower', cmap='viridis_r')
    cb = fig.colorbar(im,ax=ax)
    cb.ax.set_ylabel('Water Depth (m)',fontsize=20,rotation=-90, labelpad=30)
    cb.ax.tick_params('y',labelsize=15)
    ax.set_xlabel('Along shore (km)',fontsize=20)
    ax.set_ylabel('Cross shore (km)',fontsize=20)
    ax.tick_params('both',labelsize=15)

In [47]:
def update_animate(time,z,update_period=100):
        if time%update_period == 0:
            clear_output(wait=True)
            cem.get_value(Z, out=z)
            plot_coast(z)
            plt.title('Time Step: '+ str(time),fontsize=20)
            plt.show()

In [54]:
## need to put in the variables that get updated each time step (i think)

def run_model_loop(Number_Iterations, z ,animate=True):
    
    for time in range(Number_Iterations):
        waves.update()
        angle = waves.get_value(alpha)
        cem.set_value(alpha, angle)
        cem.set_value(Qs, qs)
        cem.update()
        if animate:
            update_animate(time,z)

        else:
            clear_output(wait=True)
            print('Time Step: ',time)
    

In [55]:
##Will run faster if animate=False 
run_model_loop(10000,z,animate=False)

Time Step:  1653


KeyboardInterrupt: 